In [35]:
import json
import os
from datetime import datetime
import requests
import sys
from pathlib import Path
from extract_markdown import extract_markdown_from_pdf, extract_from_to, count_pages
from create_embeddings import extract_pdfs_to_markdown,generate_faiss_embeddings,generate_rag_response,search_faiss

Sample Testing

In [9]:

pdf_path = r"dof\2025\01\02012025-MAT.pdf"  

num_pages = count_pages(pdf_path)
print(f"Número de páginas en el PDF: {num_pages}")



Número de páginas en el PDF: 166


In [10]:
markdown_text = extract_markdown_from_pdf(pdf_path)
print(markdown_text[:1000])  


```html
<chunk pages="1">
UNIDOS MEXICA
ESTADOS
DIARIO OFICIAL
DE LA FEDERACION
ORGANO DEL GOBIERNO CONSTITUCIONAL DE LOS ESTADOS UNIDOS MEXICANOS
No. de publicación: 001/2025
Ciudad de México, jueves 2 de enero de 2025
CONTENIDO
Secretaría de Infraestructura, Comunicaciones y Transportes
Secretaría de Desarrollo Agrario, Territorial y Urbano
Servicios de Salud del Instituto Mexicano del Seguro Social
para el Bienestar (IMSS-BIENESTAR)
Banco de México
Instituto Nacional Electoral
Tribunal Superior Agrario
Convocatorias para Concursos de Adquisiciones,
Arrendamientos, Obras y Servicios del Sector Público
Avisos
Indice en página 2
</chunk>
```


In [11]:
md_path = "extract/documento.md"  
extract_from_to(pdf_path, md_path)
print(f"Markdown guardado en: {md_path}")


Markdown guardado en: extract/documento.md


Embeddings

In [12]:

source_directory = "dof/2025"
target_directory = "extracted/2025"
extract_pdfs_to_markdown(source_directory, target_directory)

✅ Procesado: dof/2025\01\02012025-MAT.pdf → extracted/2025\01\02012025-MAT.md
✅ Procesado: dof/2025\01\03012025-MAT.pdf → extracted/2025\01\03012025-MAT.md
✅ Procesado: dof/2025\01\06012025-MAT.pdf → extracted/2025\01\06012025-MAT.md
✅ Procesado: dof/2025\01\07012025-MAT.pdf → extracted/2025\01\07012025-MAT.md
✅ Procesado: dof/2025\01\08012025-MAT.pdf → extracted/2025\01\08012025-MAT.md
✅ Procesado: dof/2025\01\09012025-MAT.pdf → extracted/2025\01\09012025-MAT.md
✅ Procesado: dof/2025\01\10012025-MAT.pdf → extracted/2025\01\10012025-MAT.md
✅ Procesado: dof/2025\01\13012025-MAT.pdf → extracted/2025\01\13012025-MAT.md
✅ Procesado: dof/2025\01\14012025-MAT.pdf → extracted/2025\01\14012025-MAT.md
✅ Procesado: dof/2025\01\15012025-MAT.pdf → extracted/2025\01\15012025-MAT.md
✅ Procesado: dof/2025\01\16012025-MAT.pdf → extracted/2025\01\16012025-MAT.md
✅ Procesado: dof/2025\01\17012025-MAT.pdf → extracted/2025\01\17012025-MAT.md
✅ Procesado: dof/2025\01\20012025-MAT.pdf → extracted/2025\01\20

TypeError: write() argument must be str, not None

In [15]:

md_directory = "extracted/2025"   # Ruta donde se guardaron los archivos .md
faiss_index_file = "faiss_index.idx"  # Archivo donde se guardará el índice FAISS

index, metadata = generate_faiss_embeddings(md_directory, faiss_index_file)

✅ Procesado: extracted/2025\01\02012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\03012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\06012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\07012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\08012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\09012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\10012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\13012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\14012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\15012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\16012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\17012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\20012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\21012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\22012025-MAT.md →

In [ ]:



# 📌 **Ejecutar la función con rutas correctas**
md_directory = "extracted/2025"  # Ajusta la ruta a tu directorio de Markdown
faiss_index_file = "faiss_index.idx"
metadata_file = "faiss_index.json"

index, metadata = generate_faiss_embeddings(md_directory, faiss_index_file, metadata_file)


✅ Procesado: extracted/2025\01\02012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\03012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\06012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\07012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\08012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\09012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\10012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\13012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\14012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\15012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\16012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\17012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\20012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\21012025-MAT.md → Embedding generado
✅ Procesado: extracted/2025\01\22012025-MAT.md →

In [37]:


# 📌 Cargar el índice FAISS y los metadatos antes de llamar a la búsqueda
faiss_index_path = "faiss_index.idx"
metadata_path = "faiss_index.json"

# Cargar el índice FAISS
index = faiss.read_index(faiss_index_path)

# 📌 Hacer una consulta
query = "Reforma fiscal en México"
results = search_faiss(query, index, metadata, top_k=5)

# 📌 Mostrar resultados
print("🔍 Resultados de la búsqueda:")
for res in results:
    print(f"\n🔹 Rank {res['rank']}")
    print(f"📄 Archivo: {res['file']}")
    print(f"📍 Ruta: {res['path']}")
    print(f"🔢 Distancia: {res['distance']}")



TypeError: search_faiss() got multiple values for argument 'top_k'

In [33]:
query = "¿Qué se entiende por CPPP en el Diario Oficial de la Federación?"
response = generate_rag_response(query)

print("Respuesta generada:")
print(response)


NameError: name 'index' is not defined